# ライトニングネットワーク中心性分析

このノートブックでは、ビットコインのライトニングネットワークにおける3つの中心性指標を分析します：

1. **媒介中心性 (Betweenness Centrality)**: ノードが他のノード間の最短経路にどれだけ頻繁に現れるか
2. **近接中心性 (Closeness Centrality)**: ノードから他のすべてのノードへの平均距離の逆数
3. **近似中心性 (Eigenvector Centrality)**: 重要なノードに接続されているノードが重要とされる

## 目的

どのノードとチャネルを開設したらルーティングがされやすくなるかを分析します。


## 1. 環境設定とインポート


In [ ]:
import sys
import os

# プロジェクトルートをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path

# ロギング設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 日本語フォント設定（必要に応じて）
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")

print("環境設定が完了しました")


## 2. データベース接続とデータ取得


In [ ]:
from src.database.connection import DatabaseConnection
from src.graph.builder import GraphBuilder

# データベース接続
db = DatabaseConnection()

# グラフ構造情報を取得
graph_info = db.get_graph_structure()
print(f"グラフ構造情報:")
print(f"  ノード数: {graph_info['node_count']}")
print(f"  エッジ数: {graph_info['edge_count']}")

# グラフを構築
graph_builder = GraphBuilder(db)
graph = graph_builder.build_graph(directed=False)

print(f"\nグラフが構築されました:")
print(f"  ノード数: {graph.number_of_nodes()}")
print(f"  エッジ数: {graph.number_of_edges()}")
print(f"  連結成分数: {nx.number_connected_components(graph)}")


## 3. 中心性計算


In [ ]:
from src.centrality.calculator import CentralityCalculator

# 中心性計算器を初期化
calculator = CentralityCalculator()

# すべての中心性指標を計算
print("中心性計算を開始します...")
centrality_scores = calculator.calculate_all(graph)

print("\n計算が完了しました:")
for name, scores in centrality_scores.items():
    print(f"  {name}: {len(scores)}個のノード")
    if scores:
        values = list(scores.values())
        print(f"    平均: {np.mean(values):.6f}")
        print(f"    最大: {np.max(values):.6f}")
        print(f"    最小: {np.min(values):.6f}")


## 4. 中心性の可視化


In [ ]:
from src.analysis.visualizer import GraphVisualizer

# 可視化器を初期化
visualizer = GraphVisualizer(output_dir='../results')

# 中心性分布を可視化
visualizer.plot_centrality_distribution(centrality_scores)

# 各中心性タイプでグラフを可視化（上位100ノード）
for centrality_type in ['betweenness', 'closeness', 'eigenvector']:
    if centrality_type in centrality_scores:
        visualizer.plot_graph_with_centrality(
            graph, 
            centrality_scores, 
            centrality_type=centrality_type,
            top_n=100
        )

# 上位ノードの比較
visualizer.plot_top_nodes_comparison(centrality_scores, top_n=20)

print("可視化が完了しました")


## 5. ルーティング可能性分析


In [ ]:
from src.analysis.analyzer import RoutingAnalyzer

# ルーティング分析器を初期化
analyzer = RoutingAnalyzer(graph, centrality_scores)

# 推奨ノードを取得（上位50ノード）
recommended_nodes = analyzer.recommend_nodes_for_channel(top_n=50)

print("推奨ノード（上位20）:")
print(recommended_nodes.head(20).to_string())

# CSVに保存
recommended_nodes.to_csv('../results/recommended_nodes.csv', index=False)
print("\n推奨ノードをCSVに保存しました")


In [ ]:
# 推奨チャネルを取得（上位100チャネル）
# 注意: 全ノードの組み合わせを計算すると時間がかかるため、
# 上位ノードのみを候補として使用
top_nodes = recommended_nodes.head(100)['node_id'].tolist()
recommended_channels = analyzer.recommend_channels(
    candidate_nodes=top_nodes,
    top_n=100
)

print("推奨チャネル（上位20）:")
print(recommended_channels.head(20).to_string())

# CSVに保存
recommended_channels.to_csv('../results/recommended_channels.csv', index=False)
print("\n推奨チャネルをCSVに保存しました")


## 6. 特徴量抽出（機械学習用）


In [ ]:
from src.ml.features import FeatureExtractor

# 特徴量抽出器を初期化
feature_extractor = FeatureExtractor(graph, centrality_scores)

# ノード特徴量を抽出
node_features_df = feature_extractor.create_node_dataframe()
print(f"ノード特徴量: {node_features_df.shape}")
print(node_features_df.head())

# エッジ特徴量を抽出
edge_features_df = feature_extractor.create_edge_dataframe()
print(f"\nエッジ特徴量: {edge_features_df.shape}")
print(edge_features_df.head())

# CSVに保存
node_features_df.to_csv('../results/node_features.csv', index=False)
edge_features_df.to_csv('../results/edge_features.csv', index=False)
print("\n特徴量をCSVに保存しました")


## 7. グラフニューラルネットワーク（GNN）による分析

このセクションでは、AWS SageMakerを使用してGNNモデルを訓練します。
詳細な手順については、`docs/GNN_CENTRALITY_ANALYSIS_GUIDE.md`と`docs/AWS_SAGEMAKER_GUIDE.md`を参照してください。

### 7.1 DGLグラフへの変換


In [ ]:
import dgl
import torch
import numpy as np

# NetworkXグラフをDGLグラフに変換
# 注意: ノードの順序を保持するため、ノードリストを明示的に指定
print("DGLグラフに変換しています...")
node_list = list(graph.nodes())  # ノードの順序を保持
dgl_graph = dgl.from_networkx(graph, node_attrs=None, edge_attrs=None)

# ノード特徴量の準備
num_nodes = dgl_graph.number_of_nodes()

# 基本特徴量（次数）
# 注意: ノードの順序が一致することを確認
degrees = torch.tensor([graph.degree(n) for n in node_list], dtype=torch.float32)
features = degrees.unsqueeze(1)

# 中心性指標を取得（ノードの順序を保持）
betweenness = torch.tensor([centrality_scores['betweenness'].get(n, 0.0) 
                            for n in node_list], dtype=torch.float32)
closeness = torch.tensor([centrality_scores['closeness'].get(n, 0.0) 
                         for n in node_list], dtype=torch.float32)
eigenvector = torch.tensor([centrality_scores['eigenvector'].get(n, 0.0) 
                           for n in node_list], dtype=torch.float32)

# 理論的注意: 中心性指標を特徴量として使用する場合、
# ラベルとして使用しないこと（データリーク防止）
# ここでは、基本特徴量（次数）のみを使用し、中心性は別途保存
# または、中心性を特徴量として使用する場合は、ラベルには使用しない

# オプション1: 基本特徴量のみを使用（推奨: データリークなし）
# features = degrees.unsqueeze(1)

# オプション2: 中心性も特徴量として使用（注意: ラベルには使用しない）
features = torch.cat([features, 
                      betweenness.unsqueeze(1),
                      closeness.unsqueeze(1),
                      eigenvector.unsqueeze(1)], dim=1)

# 特徴量の正規化（重要: 異なるスケールの特徴量を統一）
feature_mean = features.mean(dim=0, keepdim=True)
feature_std = features.std(dim=0, keepdim=True) + 1e-8  # ゼロ除算防止
features_normalized = (features - feature_mean) / feature_std

dgl_graph.ndata['feat'] = features_normalized

# 中心性指標を個別に保存（分析用）
dgl_graph.ndata['betweenness'] = betweenness
dgl_graph.ndata['closeness'] = closeness
dgl_graph.ndata['eigenvector'] = eigenvector

# ラベルの準備（ルーティング可能性）
# 注意: 中心性指標を特徴量として使用している場合、
# ラベルには別の指標（例: 実際のルーティング成功回数など）を使用すべき
# ここでは例として、中心性の組み合わせを使用（実際のデータに置き換えること）
routing_potential = (betweenness * 0.5 + closeness * 0.3 + eigenvector * 0.2)
dgl_graph.ndata['label'] = routing_potential.unsqueeze(1)

print(f"DGLグラフ情報:")
print(f"  ノード数: {dgl_graph.number_of_nodes()}")
print(f"  エッジ数: {dgl_graph.number_of_edges()}")
print(f"  特徴量次元: {features_normalized.shape[1]}")
print(f"  ラベル形状: {dgl_graph.ndata['label'].shape}")
print(f"\n注意: 中心性指標を特徴量として使用している場合、")
print(f"ラベルには実際のルーティングデータを使用することを推奨します。")


### 7.2 グラフデータの保存


In [ ]:
# グラフを保存
dgl.save_graphs('../results/lightning_graph.bin', [dgl_graph])

# メタデータの保存
import json
metadata = {
    'num_nodes': dgl_graph.number_of_nodes(),
    'num_edges': dgl_graph.number_of_edges(),
    'feature_dim': features.shape[1]
}

with open('../results/graph_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("グラフデータを保存しました: ../results/lightning_graph.bin")


### 7.3 AWS SageMakerでのGNN訓練（オプション）

**注意**: このセルを実行する前に、以下を確認してください：
1. `config/config.yaml`でAWS設定（IAMロール、S3バケット）を行ってください
2. データをS3にアップロードしてください
3. 詳細は`docs/AWS_SAGEMAKER_GUIDE.md`を参照してください


In [ ]:
# 注意: このセルを実行する前に、config/config.yamlでAWS設定を行ってください
# import boto3
# from src.ml.pipeline import GNNPipeline
# 
# # S3にデータをアップロード
# s3 = boto3.client('s3')
# bucket_name = 'your-bucket-name'  # 実際のバケット名
# 
# s3.upload_file('../results/lightning_graph.bin', bucket_name, 
#                'lightning-network-analysis/data/lightning_graph.bin')
# 
# train_data_path = f's3://{bucket_name}/lightning-network-analysis/data/lightning_graph.bin'
# 
# # GNNパイプラインを初期化
# gnn_pipeline = GNNPipeline()
# 
# # モデルの訓練
# print("GNNモデルの訓練を開始します...")
# estimator = gnn_pipeline.train_gnn_model(
#     train_data_path=train_data_path,
#     hyperparameters={
#         'hidden-dim': 64,
#         'num-layers': 2,
#         'dropout': 0.5,
#         'learning-rate': 0.01,
#         'epochs': 100,
#         'model-type': 'gcn'  # または 'gat'
#     },
#     use_dgl_container=True
# )
# 
# print("GNNモデルの訓練が完了しました")


## 8. 結果のまとめ


In [ ]:
print("=== 分析結果のまとめ ===\n")

print(f"グラフ統計:")
print(f"  ノード数: {graph.number_of_nodes()}")
print(f"  エッジ数: {graph.number_of_edges()}")
print(f"  平均次数: {sum(dict(graph.degree()).values()) / graph.number_of_nodes():.2f}")
print(f"  連結成分数: {nx.number_connected_components(graph)}")

print(f"\n中心性統計:")
for name, scores in centrality_scores.items():
    if scores:
        values = list(scores.values())
        top_node = max(scores.items(), key=lambda x: x[1])
        print(f"  {name}:")
        print(f"    最大値ノード: {top_node[0]}")
        print(f"    最大値: {top_node[1]:.6f}")

print(f"\n推奨ノード数: {len(recommended_nodes)}")
print(f"推奨チャネル数: {len(recommended_channels)}")

print("\n分析が完了しました。結果は results/ ディレクトリに保存されています。")


In [ ]:
# データベース接続を閉じる
db.close()
print("データベース接続を閉じました")
